In [ ]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/content/6D_pose"
branch = "pose_rgb"

if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url}")

if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

%cd 6D_pose

In [ ]:
%cd ..
!gdown --fuzzy https://drive.google.com/file/d/1qQ8ZjUI6QauzFsiF8EpaaI2nKFWna_kQ/view?usp=sharing -O Linemod_preprocessed.zip
!unzip Linemod_preprocessed.zip
%cd 6D_pose

In [ ]:
from google.colab import drive
from utils.load_data import mount_drive

mount_drive()

# dataset_root = "/content/drive/MyDrive/Linemod_preprocessed"
dataset_root = "/content/Linemod_preprocessed"
print(f"\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")

In [ ]:
!pip install plyfile

In [ ]:
from src.pose_pointnet.pointcloud_dataset import LineModPointCloudDataset
from src.pose_pointnet.pointnet_model import PointNetPose
from src.pose_rgb.loss import AutomaticWeightedLoss
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔥 Using device: {DEVICE}")

## 🎯 Rotation-Only Approach

**Simple and Clean:**
- **Model**: Predicts ONLY rotation (quaternion)
- **Loss**: Simple `RotationLoss` - no complex multi-task balancing needed
- **Translation**: Computed from depth + bbox using pinhole geometry (not learned)

**Why this works:**
- Depth already gives us Z → no need to learn it
- Pinhole geometry is deterministic → no need to learn X, Y from point cloud
- PointNet focuses on what it's good at: extracting shape features for rotation

**Math:**
- Depth at bbox center: $Z = \text{depth}[c_x, c_y]$
- Back-projection: $X = \frac{(c_x - c_x^{intr}) \cdot Z}{f_x}$, $Y = \frac{(c_y - c_y^{intr}) \cdot Z}{f_y}$

## ⚡ Performance Optimizations

**Optimizations applied to accelerate training:**

1. **Point reduction**: 1024 → 512 points per point cloud (~2x speed-up).
2. **Cached YAML files**: `linemod_config` caches `info.yml` and `gt.yml` instead of opening them for every iteration (~10-20x speed-up!).
    - **Critical on Google Drive**: I/O latency is extremely high, making caching essential.
3. **Torch sampling**: Used `torch.randperm` instead of `np.random.choice` (~1.5x speed-up).
4. **Mixed precision**: Automatic FP16/FP32 training with `torch.cuda.amp` (~1.5x speed-up).

**Estimated total speed-up: ~20-30x** compared to the initial version! 🚀

**Note**: The first batch may take longer to load and cache all YAML files; subsequent iterations will be significantly faster.

In [ ]:
# Crea dataset con point clouds
# Nota: Prediciamo SOLO rotazione, la traslazione viene ricavata da depth + bbox usando pinhole geometry

train_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='train',
    num_points=512,
    use_rgb=True
)

test_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='test',
    num_points=512,
    use_rgb=True
)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Visualizza un sample
sample = train_dataset[0]

print("Sample keys:", sample.keys())
print(f"Point cloud shape: {sample['point_cloud'].shape}")  # (512, 6)
print(f"Rotation shape: {sample['rotation'].shape}")        # (4,) - TENSOR
print(f"Depth Z: {sample['depth_z']:.4f} m")
print(f"\nRotation (quat, tensor): {sample['rotation']}")
print(f"Bbox info (tensor): {sample['bbox_info']}")

In [ ]:
# DataLoaders - Ottimizzati per performance
train_loader = DataLoader(
    train_dataset,
    batch_size=64,  # Aumentato da 32 per migliore GPU utilization
    shuffle=True,
    num_workers=2,  # Aumentato da 2
    pin_memory=True  # Velocizza transfer CPU->GPU
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
# Import visualization utilities
from utils.projection_utils import visualize_pose_comparison
from utils.linemod_config import get_linemod_config
from utils.projection_utils import setup_projection_utils
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path

# Config per caricare modelli 3D
config = get_linemod_config(dataset_root)
setup_projection_utils(dataset_root)

def compute_translation_from_depth(bbox_info, depth_z, cam_K, img_shape):
    """
    Compute 3D translation using pinhole geometry.
    bbox_info: tensor or numpy array
    depth_z: scalar (tensor or float)
    cam_K: tensor or numpy array
    """
    H, W = img_shape
    
    # Convert to numpy if needed
    if isinstance(bbox_info, torch.Tensor):
        bbox_info = bbox_info.numpy()
    if isinstance(cam_K, torch.Tensor):
        cam_K = cam_K.numpy()
    if isinstance(depth_z, torch.Tensor):
        depth_z = depth_z.item()
    
    cx_norm, cy_norm = bbox_info[0], bbox_info[1]
    
    # Convert normalized to pixel coordinates
    cx_pixel = cx_norm * W
    cy_pixel = cy_norm * H
    
    # Back-project using pinhole model
    fx = cam_K[0, 0]
    fy = cam_K[1, 1]
    cx_intr = cam_K[0, 2]
    cy_intr = cam_K[1, 2]
    
    Z = depth_z
    X = (cx_pixel - cx_intr) * Z / fx
    Y = (cy_pixel - cy_intr) * Z / fy
    
    return np.array([X, Y, Z])

def visualize_training_sample(model, dataset, dataset_root, checkpoint_dir, epoch, device='cuda'):
    """
    Visualizza una predizione casuale e la salva.
    """
    import shutil

    model.eval()

    # Sample casuale
    idx = np.random.randint(0, len(dataset))
    sample = dataset[idx]

    obj_id = sample['object_id']
    img_id = sample['img_id']

    # Carica immagine originale
    img_path = Path(dataset_root) / "data" / f"{obj_id:02d}" / "rgb" / f"{img_id:04d}.png"
    image = cv2.imread(str(img_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    H, W = image.shape[:2]

    # Prepare input
    point_cloud = sample['point_cloud'].unsqueeze(0).to(device)
    bbox_info = sample['bbox_info'].unsqueeze(0).to(device)

    # Inference - ONLY rotation
    with torch.no_grad():
        pred_rot = model(point_cloud, bbox_info)

    # Convert to numpy
    pred_rot = pred_rot.squeeze(0).cpu().numpy()
    gt_rot = sample['rotation'].cpu().numpy()
    cam_K = sample['cam_K'].numpy()
    
    # Compute translations from depth + bbox (same for both gt and pred)
    trans = compute_translation_from_depth(sample['bbox_info'], sample['depth_z'], cam_K, (H, W))

    # Visualizza (rotation is different, translation is same)
    img_vis = visualize_pose_comparison(
        image, obj_id, cam_K,
        gt_rot, trans,
        pred_rot, trans
    )

    # Calcola errore di rotazione
    dot_product = np.abs(np.dot(pred_rot, gt_rot))
    dot_product = np.clip(dot_product, -1.0, 1.0)
    angle_error = 2 * np.arccos(dot_product) * 180 / np.pi

    # Salva
    tmp_path = f"/tmp/vis_epoch_{epoch:03d}_obj_{obj_id:02d}.png"
    fig = plt.figure(figsize=(12, 10))
    plt.imshow(img_vis)
    plt.axis('off')
    plt.title(f"Object {obj_id} | Rotation Error: {angle_error:.2f}°")
    plt.savefig(tmp_path, bbox_inches='tight', dpi=100)
    plt.close()

    # Copia su Drive
    vis_dir = Path(checkpoint_dir) / "visualizations"
    vis_dir.mkdir(exist_ok=True)
    drive_path = vis_dir / f"epoch_{epoch:03d}_obj_{obj_id:02d}.png"
    shutil.copy(tmp_path, str(drive_path))

    print(f"✅ Saved: epoch {epoch} | Rotation Error: {angle_error:.2f}°")

    model.train()


In [ ]:
import os
import json
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt
from torch.amp import autocast, GradScaler
from src.pose_pointnet.pointnet_model import PointNetPose


# ==========================================
# HYPERPARAMETERS
# ==========================================
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
USE_MIXED_PRECISION = True

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
CHECKPOINT_DIR = f'/content/drive/MyDrive/runs/pointnet_{timestamp}'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Initialize PointNet Model - ROTATION ONLY
model = PointNetPose(input_channels=6, use_batch_norm=True).to(DEVICE)

# Loss - SIMPLE: Only Rotation Loss
from src.pose_pointnet.loss import PoseMatchingLoss
criterion = PoseMatchingLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Mixed Precision Scaler
scaler = GradScaler('cuda') if USE_MIXED_PRECISION else None

train_losses = []
val_losses = []
best_val_loss = float('inf')
best_epoch = 0

print(f"\n🔥 STARTING POINTNET ROTATION-ONLY TRAINING on {DEVICE}...")
print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
print(f"⚙️  Loss: RotationLoss (simple quaternion loss)")
print(f"⚡ Mixed Precision: {USE_MIXED_PRECISION}")

## 🎯 Rotation-Only Model Architecture

**Key Insight**: With depth available, we only need to predict **rotation**. Translation is computed directly using pinhole geometry:

$$X = \frac{(c_x - c_x^{intr}) \cdot Z}{f_x}, \quad Y = \frac{(c_y - c_y^{intr}) \cdot Z}{f_y}, \quad Z = \text{depth}[c_x, c_y]$$

**Advantages:**
- Simpler model (no translation head)
- Faster training (only rotation loss)
- More stable (depth provides ground truth Z)
- Cleaner optimization (one prediction target instead of two)

**Dataset returns:**
- `point_cloud`: Local point cloud (N×3 or N×6 with RGB)
- `rotation`: Target quaternion
- `depth_z`: Z coordinate from depth at bbox center (for inference)

In [ ]:
# ==========================================
# TRAINING LOOP - SIMPLE ROTATION LOSS
# ==========================================
for epoch in range(NUM_EPOCHS):

    # --- TRAIN PHASE ---
    model.train()
    running_train_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")

    for batch in pbar:
        # Move to device
        point_clouds = batch['point_cloud'].to(DEVICE, non_blocking=True)  # (B, N, 6)
        bbox_info = batch['bbox_info'].to(DEVICE, non_blocking=True)       # (B, 4)
        gt_rot = batch['rotation'].to(DEVICE, non_blocking=True)           # (B, 4)

        optimizer.zero_grad()

        # Forward
        if USE_MIXED_PRECISION:
            with autocast('cuda'):
                pred_rot = model(point_clouds, bbox_info)
                loss = criterion(pred_rot, gt_rot)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            pred_rot = model(point_clouds, bbox_info)
            loss = criterion(pred_rot, gt_rot)
            loss.backward()
            optimizer.step()

        running_train_loss += loss.item()
        pbar.set_postfix({'Loss': f"{loss.item():.4f}"})

    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # --- VALIDATION PHASE ---
    model.eval()
    running_val_loss = 0.0
    val_batches_limit = 50
    count_batches = 0

    with torch.no_grad():
        val_iterator = islice(test_loader, val_batches_limit)
        val_pbar = tqdm(val_iterator, total=val_batches_limit, desc="Validating")

        for batch in val_pbar:
            point_clouds = batch['point_cloud'].to(DEVICE, non_blocking=True)
            bbox_info = batch['bbox_info'].to(DEVICE, non_blocking=True)
            gt_rot = batch['rotation'].to(DEVICE, non_blocking=True)

            if USE_MIXED_PRECISION:
                with autocast('cuda'):
                    pred_rot = model(point_clouds, bbox_info)
                    loss = criterion(pred_rot, gt_rot)
            else:
                pred_rot = model(point_clouds, bbox_info)
                loss = criterion(pred_rot, gt_rot)

            running_val_loss += loss.item()
            count_batches += 1

    avg_val_loss = running_val_loss / count_batches if count_batches > 0 else 0.0
    val_losses.append(avg_val_loss)

    # --- REPORT & SAVE ---
    print(f"📊 Epoch {epoch+1}: Train Loss={avg_train_loss:.4f} | Val Loss={avg_val_loss:.4f}")

    # --- VISUALIZE RANDOM SAMPLE ---
    if (epoch + 1) % 5 == 0:
      print(f"🎨 Visualizing random validation sample...")
      visualize_training_sample(model, test_dataset, dataset_root, CHECKPOINT_DIR, epoch, DEVICE)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch + 1

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'val_loss': best_val_loss
        }, os.path.join(CHECKPOINT_DIR, "best_model.pth"))

        print(f"🏆 New Best Model! (Loss: {best_val_loss:.4f})")

    # Save last checkpoint
    if (epoch + 1) == NUM_EPOCHS:
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
        }, os.path.join(CHECKPOINT_DIR, f"checkpoint_ep{epoch+1}.pth"))

print("\n🎉 TRAINING COMPLETE!")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5,
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('PointNet Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5,
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.yscale('log')
plt.title('PointNet Training History (Log)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_DIR, 'training_history.png'), dpi=150)
plt.show()

print(f"\n📊 Training Statistics:")
print(f"   Best epoch: {best_epoch}")
print(f"   Best val loss: {best_val_loss:.6f}")
print(f"   Final train loss: {train_losses[-1]:.6f}")

# Save history
history = {
    'train_losses': [float(x) for x in train_losses],
    'val_losses': [float(x) for x in val_losses],
    'best_epoch': int(best_epoch),
    'best_val_loss': float(best_val_loss),
    'timestamp': timestamp
}

with open(os.path.join(CHECKPOINT_DIR, 'history.json'), 'w') as f:
    json.dump(history, f, indent=2)

## Visualize Point Cloud Sample

Visualizing point cloud from random point in the dataset

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Choose sample
sample = train_dataset[100]
pc = sample['point_cloud'].numpy()  # (512, 6) [x, y, z, r, g, b]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot 3D points with RGB colors
ax.scatter(pc[:, 0], pc[:, 1], pc[:, 2],
           c=pc[:, 3:6], s=2, alpha=0.6)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title(f'Point Cloud - Object {sample["object_id"]}')
plt.show()

print(f"Object ID: {sample['object_id']}")
print(f"Rotation (quat, tensor): {sample['rotation']}")
print(f"GT Translation (m, tensor): {sample['gt_translation']}")